# Association rules - 3rd laboratory

- `pandas` and [`mlxtend`](http://rasbt.github.io/mlxtend/)
- Apriori algorithm and FP-Growth 

-------
## Online Retail Data Set

In [2]:
!wget https://github.com/dbdmg/data-science-lab/raw/master/datasets/online_retail.csv -O ords.csv

--2022-04-15 09:56:31--  https://github.com/dbdmg/data-science-lab/raw/master/datasets/online_retail.csv
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/online_retail.csv [following]
--2022-04-15 09:56:31--  https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/online_retail.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45580638 (43M) [text/plain]
Saving to: ‘ords.csv’

ords.csv            100%[===================>]  43,47M  25,9MB/s    in 1,7s    

2022-04-15 09:56:33 (25,9 MB/s) - ‘ords.csv’ saved [45580638/45580638]



In [1]:
import numpy as np
import pandas as pd

In [2]:
import csv

def read_csv(file_name:str) -> list:
    """Read and return a list of list of the input csv file

    Args:
        file_name (str): input CSV file

    Returns:
        list: list of list of the input file, one list for each line
        in the csv file
    """
    with open(file_name, "r") as f:
        lol = []
        for cols in csv.reader(f):
            lol.append(cols)
    return lol

In [3]:
ords_lol = np.array(read_csv('ords.csv'))

In [4]:
def indexes_to_delete(ary:np.array) -> list:
    """Return a list containing the indexes 
    that belong to a cancelled transaction

    Args:
        ary (np.array): input array

    Returns:
        list: output list containing indexes
    """
    indexes_to_delete = []
    for transaction_id in range(ary.shape[0] - 1):
        if ary[transaction_id,0][0] == 'C':
            indexes_to_delete.append(transaction_id)
        else:
            continue    
    return indexes_to_delete


ids_to_delete = indexes_to_delete(ords_lol)


In [5]:
filtered_ords = np.delete(ords_lol, ids_to_delete, axis=0)
ids_to_delete = None

In [6]:
indexes_to_delete(filtered_ords) # return empty list

[]

In [32]:
invoices = sorted(set(filtered_ords[:,0]))

transactions = {id:[] for id in invoices}

for t in filtered_ords[1:]:
    id = t[0]
    element = t[2]
    transactions[id].append(element)    
    

In [ ]:
transactions.values()

In [39]:
number_of_itemsets = len(transactions.values()) 
set_of_items = sorted(set(filtered_ords[1:,2]))
matrix = np.zeros(shape=(number_of_itemsets, len(set_of_items)))

In [50]:
arys = [] #list of arrays, 1 array for each transaction
for t in transactions.values():  
    ary = np.zeros(len(set_of_items))
    for transaction_item in t:
        id = 0
        for item in set_of_items:
            if transaction_item == item:
                ary[id] = 1
                id += 1
            else:
                id += 1
                continue
    arys.append(ary)

In [49]:
len(set_of_items)

4208

In [53]:
arys[0].sum()

7.0

In [56]:
list(transactions.values())[0]

['WHITE HANGING HEART T-LIGHT HOLDER',
 'WHITE METAL LANTERN',
 'CREAM CUPID HEARTS COAT HANGER',
 'KNITTED UNION FLAG HOT WATER BOTTLE',
 'RED WOOLLY HOTTIE WHITE HEART.',
 'SET 7 BABUSHKA NESTING BOXES',
 'GLASS STAR FROSTED T-LIGHT HOLDER']

In [58]:
arys = np.array(arys)

In [61]:
df = pd.DataFrame(data = arys, columns=set_of_items)

In [63]:
df.head()

,,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
